In [10]:
%cd /home/burak/repos/smartrag

/home/burak/repos/smartrag


/home/burak/repos/smartrag/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
from datasets import load_dataset
import json
import pandas as pd
import wandb
import os

In [12]:
"""%%capture
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="smartrag",
    name="ambiguous_query_generation",
    save_code=False,
)  # start a new wandb run to track this script"""

'%%capture\n# start a new wandb run to track this script\nwandb.init(\n    # set the wandb project where this run will be logged\n    project="smartrag",\n    name="ambiguous_query_generation",\n    save_code=False,\n)  # start a new wandb run to track this script'

In [13]:
from disambiguation_methods.generate_abbreviations import generate_ambiguous_queries
from disambiguation_methods.api_suggestion import get_abbreviation_suggestions as get_abbreviation_suggestions_api
from disambiguation_methods.llm_suggestion import get_abbreviation_suggestions as get_abbreviation_suggestions_llm
from disambiguation_methods.ambiguity_extractor import extract_ambiguities
from disambiguation_methods.intent_extractor import extract_intent
from disambiguation_methods.domain_extractor import extract_domains
from disambiguation_methods.dtype_extractor import extract_dtypes

In [14]:
"""!wget https://storage.googleapis.com/ai2i/strategyqa/data/strategyqa_dataset.zip -O strategyqa.zip
!unzip strategyqa.zip -d dataset
!wget https://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz -O triviaqa.tar.gz
!tar xzvf triviaqa.tar.gz
!wget https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip -O ambignq.zip
!unzip ambignq.zip -d dataset
"""

'!wget https://storage.googleapis.com/ai2i/strategyqa/data/strategyqa_dataset.zip -O strategyqa.zip\n!unzip strategyqa.zip -d dataset\n!wget https://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz -O triviaqa.tar.gz\n!tar xzvf triviaqa.tar.gz\n!wget https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip -O ambignq.zip\n!unzip ambignq.zip -d dataset\n'

In [15]:
n_queries = 75
top_n = 10
dataset_name = "ambiguity/meadow"
experiment = "ambiguous"
# experiment = "generated"
# experiment = "unfiltered"
GENERATE = False

In [28]:
if GENERATE:
    df = generate_ambiguous_queries(n_queries=n_queries, temperature=1.0)
    df.to_csv(f"./data/{dataset_name}_{experiment}.csv")
else:
    if not os.path.exists(f"./data/{dataset_name}_{experiment}.csv"):
        # dataset = pd.DataFrame(json.loads(open("dataset/strategyqa_train.json").read()))
        # dataset = pd.DataFrame(json.loads(open("triviaqa-unfiltered/unfiltered-web-dev.json").read())["Data"])
        # dataset = pd.DataFrame(json.loads(open("./dataset/dev_light.json").read()))
        # dataset = pd.DataFrame(load_dataset("rajpurkar/squad")["train"])
        dataset = pd.DataFrame(load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"])
        # dataset = pd.DataFrame(load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"])
        # dataset.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)
        # with open("dev.jsonl") as f:
        #  dataset_boolq = pd.json_normalize(map(lambda x: json.loads(x), f.readlines()))
        n_sample = 150
        df = dataset.sample(n_sample, random_state=66).reset_index(drop=True)
        df.to_csv(f"./data/{dataset_name}_{experiment}.csv", index=False)
    else:
        df = pd.read_csv(f"./data/{dataset_name}_{experiment}.csv", sep=",")

In [29]:
# extract_ambiguities(df)
extract_domains(df)
extract_dtypes(df)
extract_intent(df)

# get_abbreviation_suggestions_api(df, top_n)
# get_abbreviation_suggestions_llm(df, top_n)

  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [02:41<00:00,  1.08s/it, Informational: ]


In [30]:
# table = wandb.Table(dataframe=df)
# wandb.log({f"{dataset_name}_ambiguous_top{top_n}+DOMAIN+LLM+Intent": table})
df.to_csv(f"./data/{dataset_name}_{experiment}+DOMAIN+Intent+Dtype.csv", index=False)

In [19]:
df.to_csv("a.csv")